# Testing ChatESG

In [2]:
import requests
import json

In [3]:
TEST_UID = "#test-uid"
MODE = "local"

if MODE=="local":
    base_url = 'http://localhost:8000'

# Health check
requests.get(f"{base_url}/health").content, base_url

(b'{"status":"OK"}', 'http://localhost:8000')

### Chat

In [101]:
headers_chat = {
    "Content-Type": "application/json",
    "test-uid": TEST_UID
}

chat_payload = {
    "src": "skripsi",
    "model": "gpt-4o-mini",
    # "model": 'gemma-2-27b-it',
    # "bsid": "7f79f395c3564549be52",
    "retriever": 'similarity',
    "top_k": 4,
    "inputs": [
        # {"type": "text", "role": "user", "content": "sebutkan nama presiden komisaris dari perusahaan"}
        {"type": "text", "role": "user", "content": "kalau presiden direkturnya siapa?"}
    ]
}

response = requests.post(f"{base_url}/api/v1/chat", headers=headers_chat, data=json.dumps(chat_payload), stream=True)
for line in response.iter_lines():
    if line:
        chunk = json.loads(line.decode('utf-8'))
        print("Chunk received:", chunk)
chunk

Chunk received: {'done': False, 'tools': ['Mengambil informasi relevan dari laporan keberlanjutan'], 'responses': [], 'sources': None, 'bsid': 'aca43de8ae004a46a69c', 'bcid': 'ebdce24d-cff8-4676-aaad-b678393b3183', 'metadata': None}
Chunk received: {'done': False, 'tools': ['Mengambil informasi relevan dari laporan keberlanjutan'], 'responses': [''], 'sources': None, 'bsid': 'aca43de8ae004a46a69c', 'bcid': 'ebdce24d-cff8-4676-aaad-b678393b3183', 'metadata': None}
Chunk received: {'done': False, 'tools': ['Mengambil informasi relevan dari laporan keberlanjutan'], 'responses': ['Pres'], 'sources': None, 'bsid': 'aca43de8ae004a46a69c', 'bcid': 'ebdce24d-cff8-4676-aaad-b678393b3183', 'metadata': None}
Chunk received: {'done': False, 'tools': ['Mengambil informasi relevan dari laporan keberlanjutan'], 'responses': ['Presiden'], 'sources': None, 'bsid': 'aca43de8ae004a46a69c', 'bcid': 'ebdce24d-cff8-4676-aaad-b678393b3183', 'metadata': None}
Chunk received: {'done': False, 'tools': ['Mengamb

{'done': True,
 'tools': ['Mengambil informasi relevan dari laporan keberlanjutan'],
 'responses': ['Presiden Direktur PT Astra International Tbk adalah Djony Bunarto Tjondro.'],
 'sources': [{'url': 'https://www.astra.co.id/',
   'pages': [60, 61, 30, 59],
   'snippet': None,
   'related_indicators': None}],
 'bsid': 'aca43de8ae004a46a69c',
 'bcid': 'ebdce24d-cff8-4676-aaad-b678393b3183',
 'metadata': None}